In [33]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from sklearn.model_selection import train_test_split


In [34]:
# Train features
df_Xtr = pd.read_csv("//home/safa/Downloads/kernel-methods-ammi-2020 (2)/Xtr.csv")

df_Xtr_mat100 = pd.read_csv("/home/safa/Downloads/kernel-methods-ammi-2020 (2)/Xtr_mat100.csv", header=None, sep=' ')

# Train labels
df_Ytr = pd.read_csv("/home/safa/Downloads/kernel-methods-ammi-2020 (2)/Ytr.csv")


# Test features
df_Xte = pd.read_csv("/home/safa/Downloads/kernel-methods-ammi-2020 (2)/Xte.csv")


df_Xte_mat100 = pd.read_csv("/home/safa/Downloads/kernel-methods-ammi-2020 (2)/Xte_mat100.csv", header=None, sep=' ')


In [67]:
df_Xtr.head()

,Id,seq
0,0,GAGGGGCTGGGGAGGGGGCTGGCCCAGAGGCACCAGACTCTGCAGA...
1,1,CGGCCTGGGGGCCACATGTGAGTGCTTACCTGTGTGGGGATGAGGG...
2,2,GACAACGCCGCTGTCAGCCGCCTTCGACTCACCTGGGAGGTGATGA...
3,3,GCCTCCCTTGGCACCACGGGAGACCAGTTTTGGAGGGGCGGGGCTG...
4,4,GCACTACTACACCCATTGCTGTAATAGTAAGTGCCGGTGCCTTCAC...


In [35]:
## We need to parallize its computation
def GaussKernel(X1, X2, sigma = 1):
    n, m = X1.shape[0], X2.shape[0]
    K = np.zeros((n,m))
    
    for i in range(n):
        for j in range(m):
            K[i,j] = np.exp(-((np.linalg.norm(X1[i]-X2[j]))**2)/(2*sigma**2))
    return K

In [37]:
#### We compute the Kernel matrix for each of the tree train sets and we save them in Kernel_Matrix directory


# Tranforming into numpy.arrays -- train
Xtr_mat100 = np.array(df_Xtr_mat100)


# Tranforming into numpy.arrays -- test
Xte_mat100 = np.array(df_Xte_mat100)


# Transforming the labels into numpy.arrays 
y = np.array(df_Ytr)[:,1]


In [38]:
# We should parallelize this computation
K_Xtr = GaussKernel(Xtr_mat100, Xtr_mat100)
#np.save("Kernel_Matrix/gaussian_kernel_Xtr.npy",K_Xtr)


#### 2.2. Implement SVM with gaussian kernel

In [40]:
!pip3 install cvxopt

    100% |████████████████████████████████| 11.7MB 175kB/s ta 0:00:011


In [41]:
import cvxopt
from cvxopt import matrix

def solve_dual_SVM(K,y, lambda_ = 1):
    n = K.shape[0] 
    G = np.vstack((np.diag(y),-np.diag(y)))
    h = np.vstack(((1/(2*lambda_*n))*np.ones((n,1)),np.zeros((n,1))))

    P = K
    q = -y.reshape(-1,1)
    #P = .5 * (P + P.T)  # make sure P is symmetric
    args = [matrix(P), matrix(q)]

    args.extend([matrix(G), matrix(h)])

    sol = cvxopt.solvers.qp(*args) 

    return np.array(sol['x']).reshape((P.shape[1],))

In [44]:
alpha_star = solve_dual_SVM(K_Xtr,2*y-1., lambda_= 0.000001)


     pcost       dcost       gap    pres   dres
 0:  2.7467e+05 -1.4336e+07  1e+07  3e-17  4e-12
 1: -1.3013e+05 -1.9370e+06  2e+06  1e-16  5e-12
 2: -2.4813e+05 -5.7461e+05  3e+05  1e-16  6e-12
 3: -3.1592e+05 -4.6411e+05  1e+05  2e-16  8e-12
 4: -3.4657e+05 -4.1970e+05  7e+04  2e-16  7e-12
 5: -3.6326e+05 -3.9692e+05  3e+04  2e-16  7e-12
 6: -3.6986e+05 -3.8860e+05  2e+04  2e-16  7e-12
 7: -3.7342e+05 -3.8391e+05  1e+04  2e-16  8e-12
 8: -3.7611e+05 -3.8032e+05  4e+03  2e-16  8e-12
 9: -3.7750e+05 -3.7853e+05  1e+03  2e-16  9e-12
10: -3.7790e+05 -3.7803e+05  1e+02  2e-16  8e-12
11: -3.7795e+05 -3.7797e+05  2e+01  2e-16  9e-12
12: -3.7796e+05 -3.7796e+05  1e+00  2e-16  9e-12
13: -3.7796e+05 -3.7796e+05  2e-02  2e-16  9e-12
Optimal solution found.


### 2.3. Predictions on test set

In [45]:
# We should parallelize this computation
K_Xte = GaussKernel(Xtr_mat100, Xte_mat100)
#np.save("Kernel_Matrix/gaussian_kernel_Xte.npy",K_Xte)

In [46]:
prediction = alpha_star.reshape(-1,1).T.dot(K_Xte)
prediction[prediction>0] = 1
prediction[prediction <0] = 0

### 2.4. Evaluation

In [47]:
train_prediction = (np.sign(alpha_star.reshape(-1,1).T.dot(K_Xtr))+1)/2
print('Train Accuracy :',1- np.abs(train_prediction - y).sum()/y.shape[0])

Train Accuracy : 0.6575


In [71]:
print(train_prediction)

[[1. 0. 0. ... 0. 1. 0.]]


###  Sickit-learn

In [51]:
from sklearn.svm import SVC

In [66]:
clf = SVC(gamma='auto')
clf.fit(Xtr_mat100, y)
predciton_sk = clf.predict(Xte_mat100)


In [70]:
print(predciton_sk)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

### Spectrum Kernel
#### Define the kernel

In [74]:
def getSubString(mString, spectrum):
    
    dictionnary = {}
    for i in range(len(mString)-spectrum+1):
        if mString[i:i+spectrum] in dictionnary:
            dictionnary[mString[i:i+spectrum]] += 1
        else:
            dictionnary[mString[i:i+spectrum]] = 1
    return dictionnary

def SpectrumKernelFunction(mString1, mString2, spectrum):
    dictionnary1 = getSubString(mString1, spectrum)
    dictionnary2 = getSubString(mString2, spectrum)
    
    kernel = 0
    for i in dictionnary1:
        if i in dictionnary2:
            kernel += dictionnary1[i] * dictionnary2[i]
    return kernel

## We should improve this function to take less time
def SpectrumKernelMatrix_train(serie,spectrum):
    n = serie.shape[0]
    K = np.zeros((n,n))
    for i,seq1 in enumerate(serie):
        for j,seq2 in enumerate(serie):
            if i <= j :
                K[i,j] = SpectrumKernelFunction(seq1, seq2, spectrum)
                K[j,i] = K[i,j]
    return(K)

def SpectrumKernelMatrix_test(serie_train, serie_test, spectrum):
    n = serie_train.shape[0]
    m = serie_test.shape[0]
    K = np.zeros((n,m))
    for i,seq1 in enumerate(serie_test):
        for j,seq2 in enumerate(serie_train):
            K[j,i] = SpectrumKernelFunction(seq1, seq2, spectrum)
    return(K)


We compute the Kernel matrix for each of the tree train sets and we save them in Kernel_Matrix directory

Solve the standard weighted kernel logisitc regression (WKLR) problem

In [75]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

### We need to improve this ####
def sqrtMatrix(W):
    # To compute the square root of a symetric positive matrix
    D,V = np.linalg.eig(W)
    return np.dot(np.dot(V,np.diag(np.sqrt(D))),np.linalg.inv(V))

def solveWKRR(K,W,z,lambda_):
    n = K.shape[0]
    W_sqrt = np.real(sqrtMatrix(W))
    
    temp = np.dot(np.dot(W_sqrt,K),W_sqrt) +  n*lambda_*np.eye(n)
    return  np.dot(W_sqrt,np.linalg.solve(temp,np.dot(W_sqrt,z)))

def solveKLR(K,y,alpha0,lambda_ = 1,itermax = 30, eps =1e-6):
    n = K.shape[0]
    
    iter_ = 0
    last_alpha = 10*alpha0 + np.ones(alpha0.shape)
    alpha = alpha0
    
    while (iter_< itermax) and (np.linalg.norm(last_alpha-alpha)>eps) :         
        print(iter_,np.linalg.norm(last_alpha-alpha))
        last_alpha = alpha
        m = np.dot(K,alpha)
        P = np.zeros((n,1))
        W = np.zeros((n,n))
        z = np.zeros((n,1))
        for i in range(n):
            P[i,0] = -sigmoid(-y[i]*m[i])
            W[i,i] = sigmoid(m[i])*sigmoid(-m[i])
            z[i,0] = m[i] - (P[i,0]*y[i])/W[i,i]
        alpha = solveWKRR(K,W,z,lambda_)
        iter_ = iter_ +1
        
      
    return alpha

In [76]:
K = K_Xtr
y_0 = y.reshape((y.shape[0],1))
y_0 = 2*y_0-1
n = y_0.shape[0]
alpha0 = np.zeros((n,1))
alpha_0 = solveKLR(K,y_0,alpha0,10) 

0 44.721359549995796
1 0.001118033181728877


In [77]:
def sign(x):
    y = x
    n = x.shape[0]
    for i in range(n):
        if x[i,0] > 0:
            y[i,0] = 1
        else:
            y[i,0] = 0
    return y

print('Accuracy:',np.linalg.norm(1-sign(np.dot(K,alpha_0))+y_0,1)/y_0.shape[0])

Accuracy: 0.998


In [83]:
prediction = pd.DataFrame(prediction.astype(int), columns=['Bound'])

df.to_csv("Prediction/tf_idf_SVM.csv",index = False)

FileNotFoundError: [Errno 2] No such file or directory: 'Prediction/tf_idf_SVM.csv'